<a href="https://colab.research.google.com/github/SRITHU2007/blank-app/blob/main/music__project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

project_path = "/content/drive/MyDrive/AI_Music_Project"
os.makedirs(project_path, exist_ok=True)

print("Project folder ready!")


Project folder ready!


In [3]:
os.listdir("/content/drive/MyDrive/AI_Music_Project")


['Beethoven — Moonlight Sonata [MIDIfind.com].mid',
 'Beethoven — Fur Elise [MIDIfind.com].mid',
 'Beethoven — Moonlight Sonata (Third Movement) [MIDIfind.com].mid',
 'Beethoven — Ode to Joy [MIDIfind.com].mid',
 'fur_elise_3.mid',
 'Beethoven — Fur Elise (4) [MIDIfind.com].mid',
 "Beethoven — Shepard's Song Symphonie Pastorale [MIDIfind.com].mid",
 'Beethoven — Hymn [MIDIfind.com] (1).mid',
 'Beethoven — Andante Acoustic [MIDIfind.com].mid',
 'Beethoven — Sonata Pathetique [MIDIfind.com].mid']

In [4]:
!pip install music21
!pip install tensorflow

In [5]:
import numpy as np
import os
import glob
import tensorflow as tf
from music21 import converter, instrument, note, chord


In [6]:
import glob

midi_path = "/content/drive/MyDrive/AI_Music_Project"

files = glob.glob(midi_path + "/*.mid") + glob.glob(midi_path + "/*.midi")

print("Number of MIDI files:", len(files))
print(files)



Number of MIDI files: 10
['/content/drive/MyDrive/AI_Music_Project/Beethoven — Moonlight Sonata [MIDIfind.com].mid', '/content/drive/MyDrive/AI_Music_Project/Beethoven — Fur Elise [MIDIfind.com].mid', '/content/drive/MyDrive/AI_Music_Project/Beethoven — Moonlight Sonata (Third Movement) [MIDIfind.com].mid', '/content/drive/MyDrive/AI_Music_Project/Beethoven — Ode to Joy [MIDIfind.com].mid', '/content/drive/MyDrive/AI_Music_Project/fur_elise_3.mid', '/content/drive/MyDrive/AI_Music_Project/Beethoven — Fur Elise (4) [MIDIfind.com].mid', "/content/drive/MyDrive/AI_Music_Project/Beethoven — Shepard's Song Symphonie Pastorale [MIDIfind.com].mid", '/content/drive/MyDrive/AI_Music_Project/Beethoven — Hymn [MIDIfind.com] (1).mid', '/content/drive/MyDrive/AI_Music_Project/Beethoven — Andante Acoustic [MIDIfind.com].mid', '/content/drive/MyDrive/AI_Music_Project/Beethoven — Sonata Pathetique [MIDIfind.com].mid']


In [7]:
from music21 import converter, instrument, note, chord

notes = []

for file in files:
    midi = converter.parse(file)

    parts = instrument.partitionByInstrument(midi)

    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

print("Total notes extracted:", len(notes))
print("Unique notes:", len(set(notes)))


Total notes extracted: 8632
Unique notes: 176


In [8]:
unique_notes = sorted(set(notes))

note_to_int = {note: number for number, note in enumerate(unique_notes)}
int_to_note = {number: note for number, note in enumerate(unique_notes)}

print("Vocabulary size:", len(unique_notes))


Vocabulary size: 176


In [9]:
sequence_length = 100

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]

    network_input.append([note_to_int[n] for n in sequence_in])
    network_output.append(note_to_int[sequence_out])

print("Total training sequences:", len(network_input))


Total training sequences: 8532


In [10]:
import numpy as np

network_input = np.array(network_input)
network_output = np.array(network_output)

print("Input shape before reshape:", network_input.shape)


Input shape before reshape: (8532, 100)


In [11]:
n_patterns = network_input.shape[0]

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

print("Input shape after reshape:", network_input.shape)


Input shape after reshape: (8532, 100, 1)


In [12]:
network_input = network_input / float(len(unique_notes))


In [13]:
print("Min:", np.min(network_input))
print("Max:", np.max(network_input))


Min: 0.0
Max: 0.9943181818181818


In [14]:
print(network_input.shape)
print(network_output.shape)


(8532, 100, 1)
(8532,)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(50, 1)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,961 (66.25 KB)

 Trainable params: 16,961 (66.25 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(network_input, network_output, epochs=20, batch_size=32)


Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 16317.9463
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 11555.1660
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 8685.1172
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 6542.8628
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 4937.0967
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 3770.9941
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 2984.0369
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 2428.1909
Epoch 9/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 2087.9871
Epoch 10/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1830.5708
Epoch 11/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1824.3894
Epoch 12/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1761.9949
Epoch 13/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1660.4106
Epoch 14/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1711.9141
Epoch 15/20
2

In [17]:
from tensorflow.keras.utils import to_categorical

network_output = to_categorical(network_output)

print("New output shape:", network_output.shape)


New output shape: (8532, 176)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import backend as K

K.clear_session()


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(Dropout(0.3))

model.add(LSTM(512))
model.add(Dropout(0.3))

model.add(Dense(len(unique_notes), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 176)            │        90,288 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,242,160 (12.37 MB)

 Trainable params: 3,242,160 (12.37 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
history = model.fit(network_input, network_output,
                    epochs=20,
                    batch_size=32)


Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - loss: 4.5030
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 4.3174
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 4.3198
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 4.3364
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 4.3374
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 4.3250
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 4.3173
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 4.2368
Epoch 9/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 4.1396
Epoch 10/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 4.1024
Epoch 11/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 4.0488
Epoch 12/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 3.9631
Epoch 13/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 3.8233
Epoch 14/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 3.8029
Epoch 15/20
267/267 ━━━━━━━━━━━━

In [21]:
import numpy as np

def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)


In [23]:
import numpy as np

prediction_output = []

# Initialize pattern before the loop
start = np.random.randint(0, len(network_input) - 1)
pattern = network_input[start]
# pattern is already reshaped in network_input to (sequence_length, 1)
# The line below ensures it has the correct batch dimension (1, sequence_length, 1)
pattern = pattern.reshape(1, sequence_length, 1)


for note_index in range(200):

    prediction = model.predict(pattern, verbose=0)

    index = sample_with_temperature(prediction[0], temperature=0.8)

    result = int_to_note[index]
    prediction_output.append(result)

    pattern = np.append(pattern[0], [[index / float(len(unique_notes))]], axis=0)
    pattern = pattern[1:]
    pattern = pattern.reshape(1, sequence_length, 1)


In [24]:
from music21 import stream, note, chord

offset = 0
output_notes = []

for pattern in prediction_output:

    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.offset = offset
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        output_notes.append(new_chord)

    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        output_notes.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='beethoven_ai.mid')


'beethoven_ai.mid'

In [25]:
from google.colab import files
files.download("beethoven_ai.mid")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
def generate_music(model, network_input, int_to_note,
                   unique_notes,
                   temperature=0.7,
                   length=500,
                   output_name="output.mid"):

    import numpy as np
    from music21 import stream, note, chord

    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
    pattern = pattern.reshape(1, pattern.shape[0], 1)

    prediction_output = []

    for note_index in range(length):
        prediction = model.predict(pattern, verbose=0)
        index = sample_with_temperature(prediction[0], temperature)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern[0],
                            [[index / float(len(unique_notes))]],
                            axis=0)
        pattern = pattern[1:]
        pattern = pattern.reshape(1, pattern.shape[0], 1)

    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.offset = offset
                notes.append(new_note)
            output_notes.append(chord.Chord(notes))
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', output_name)

    print("Generated:", output_name)


In [27]:
print("Select Music Type:")
print("1 - Classical")
print("2 - Romantic")
print("3 - Both")

choice = input("Enter choice: ")

print("\nSelect Creativity Level:")
print("1 - Low")
print("2 - Medium")
print("3 - High")

temp_choice = input("Enter choice: ")

if temp_choice == "1":
    temperature = 0.5
elif temp_choice == "2":
    temperature = 0.7
else:
    temperature = 1.0

print(f"\nGenerating music with creativity level: {temperature}")
generate_music(model, network_input,
               int_to_note,
               unique_notes,
               temperature,
               500,
               "generated_music.mid")


Select Music Type:
1 - Classical
2 - Romantic
3 - Both
Enter choice: 2

Select Creativity Level:
1 - Low
2 - Medium
3 - High
Enter choice: 2

Generating music with creativity level: 0.7
Generated: generated_music.mid


In [28]:
from google.colab import files
files.download("generated_music.mid")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>